Experiment with OpenCV for finding the pool table edges within an image

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import math
from Utils import *
import copy

In [ ]:
# Do the calibration
dirImages = "AmcrestCamera/Session3"
dirCalImages = dirImages + "/checkerboards"

GenerateCalibration(dirCalImages, False)

# Load the test image
files, dir = FindImageFilesAndDir(subdir=dirImages)
filename = files[0]
filename = "{0}/empty43.jpg".format(dir)
print("Loading file {0}".format(filename))
imageOriginal = cv.imread(filename)
imageOriginal = CalibrateImage(imageOriginal)

ImgShow(imageOriginal,150)
#print(calROI)

In [ ]:
#Find the edges using flood, start with a blur
blur = int(85)
imageBlur = cv.GaussianBlur(imageOriginal, (blur,blur), cv.BORDER_DEFAULT)

imageFeltFlood = imageBlur.copy()
cols = imageFeltFlood.shape[1]
rows = imageFeltFlood.shape[0]
seed = (int(cols / 2), int(rows / 2)) # a point in the middle

imageFeltFloodMaskBrdr = np.zeros((rows+2,cols+2,1), np.uint8)
tolerance = 38.0
diff = (tolerance,tolerance,tolerance)
flags = 8 | cv.FLOODFILL_FIXED_RANGE | 255 << 8
res = cv.floodFill(imageFeltFlood, imageFeltFloodMaskBrdr, seed, (0,0,255),diff,diff, flags)

imageFeltFloodMask = imageFeltFloodMaskBrdr[1:rows+1, 1:cols+1]
print("imageFeltFlood shape         = {0}".format(imageFeltFlood.shape))
print("imageFeltFloodMaskBrdr shape = {0}".format(imageFeltFloodMaskBrdr.shape))
print("imageFeltFloodMask shape    = {0}".format(imageFeltFloodMask.shape))
ImgShow([imageBlur, imageFeltFlood, imageFeltFloodMask], 220)

In [ ]:
# Brendan's Method
#   Scale the felt up

# Dilate the image 
kernel = np.ones((12,12), np.uint8)
imageFeltFloodMaskDilated = cv.dilate(imageFeltFloodMask, kernel, iterations=8)

# Scale up
factor = 1.3
sizeNew = (int(imageFeltFloodMaskDilated.shape[1] * factor), int(imageFeltFloodMaskDilated.shape[0] * factor))
print("sizeNew={0}".format(sizeNew))
imageExpanded = cv.resize(imageFeltFloodMaskDilated, dsize=sizeNew, interpolation=cv.INTER_AREA)
print("imageExpanded.shape={0}".format(imageExpanded.shape))
print("imageFeltFloodMask.shape={0}".format(imageFeltFloodMask.shape))

# Extract the middle ROI back to the same dimensions
centerX = int(imageExpanded.shape[1] / 2)
centerY = int(imageExpanded.shape[0] / 2)
startY = centerY - int(imageFeltFloodMask.shape[0] / 2)
startX = centerX - int(imageFeltFloodMask.shape[1] / 2)
endY = imageFeltFloodMask.shape[0] + startY
endX = imageFeltFloodMask.shape[1] + startX
print("startY={0} endY={1} startY={2} startX={3}".format(startY, endY, startX, endX))
imageExpandedROI = imageExpanded[startY:endY, startX:endX]

# Apply the mask to the original image using bitwise-and
ret,imageWoodRailsMask = cv.threshold(imageExpandedROI,127,255,cv.THRESH_BINARY)
imageWoodRailsMaskRGB = cv.cvtColor(imageWoodRailsMask, cv.COLOR_GRAY2BGR)
ret,imageWoodRailsMaskRGB = cv.threshold(imageWoodRailsMaskRGB,127,255,cv.THRESH_BINARY)
imageTablePlus = cv.bitwise_and(imageOriginal, imageWoodRailsMaskRGB)

ImgShow([imageExpandedROI, imageTablePlus],222)


In [ ]:
# Attempt to find four corners of the table

#filters image bilaterally and displays it
bilatImg = cv.bilateralFilter(imageTablePlus, d=5, sigmaColor=175, sigmaSpace=75)

#finds edges of bilaterally filtered image and displays it
imageCanny = cv.Canny(bilatImg, 75, 180)

#gets contours (outlines) for shapes and sorts from largest area to smallest area
contours, hierarchy = cv.findContours(imageCanny, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv.contourArea, reverse=True)

# drawing red contours on the image
imageContours = imageOriginal.copy()
for con in contours:
    cv.drawContours(imageContours, con, -1, (0, 0, 255), 3)

# find the perimeter of the first closed contour
perim = cv.arcLength(contours[0], True)
# setting the precision
epsilon = 0.02*perim
# approximating the contour with a polygon
approxCorners = cv.approxPolyDP(contours[0], epsilon, True)
# check how many vertices has the approximate polygon
approxCornersNumber = len(approxCorners)
print("Number of approximated corners: ", approxCornersNumber)

# can also be used to filter before moving on [if needed]
# i.e. if approxCornersNumber== 4:

# printing the position of the calculated corners
print("Coordinates of approximated corners:\n", approxCorners)

cv.polylines(imageContours, approxCorners, True, (0,255,0), 6)

ImgShow([bilatImg, imageCanny, imageContours],220)

Find the line segments

In [ ]:
# HoughLinesP for detection
img = imageCanny

cols = imageOriginal.shape[1]
rows = imageOriginal.shape[0]
shortSide = min(cols, rows)
minLineLen = shortSide / 16

rho = 1  # distance resolution in pixels of the Hough grid
theta = np.pi / 180  # angular resolution in radians of the Hough grid
threshold = 35  # minimum number of votes (intersections in Hough grid cell)
min_line_length = 150  # minimum number of pixels making up a line
min_line_length = minLineLen
max_line_gap = 60  # maximum gap in pixels between connectable line segments

# Run Hough on edge detected image
# Output "lines" is an array containing endpoints of detected line segments
tableLines = cv.HoughLinesP(img, rho, theta, threshold, np.array([]),
                    min_line_length, max_line_gap)
print("tableLines count = " + str(len(tableLines)))

# For visualization, draw the lines onto a copy of the image
imageDrawnLines = np.zeros(img.shape[:3],np.uint8)  # creating a blank to draw lines on

pts = []
for line in tableLines:
    for x1,y1,x2,y2 in line:
        pt1 = [x1,y1]
        pt2 = [x2,y2]
        cv.line(imageDrawnLines,pt1,pt2,(255,0,0),3)
        pts.append(pt1)
        pts.append(pt2)

# calculate points for each contour

# Make a simple square to test, with a point in the middle
ptsSquare = [
    [0,0],
    [100,0],
    [100,100],
    [0,100],
    [50,50],
]
#pts = ptsSquare
#print("ptsSquare",ptsSquare)
#print("pts",pts)

# creating convex hull object for each contour
hull = cv.convexHull(np.array(pts), False)
#print("hull=", hull)

# Convert hull to points list
ptsHullRaw = []
for cell in hull:
    ptsHullRaw.append(cell[0])
ptsHull = np.array(ptsHullRaw, dtype=np.int32)
ptsHull.reshape((-1,1,2))

imageConvexMask = np.zeros(img.shape[:3],np.uint8) 
cv.fillConvexPoly(imageConvexMask, np.array(ptsHull, dtype=np.int32), (255))

# Subtract out the felt, mask out the internal felt by using a bitwise-and
imageFeltFloodMaskInv = cv.bitwise_not(imageFeltFloodMask)
imageWoodRailsMask = cv.bitwise_and(imageExpandedROI, imageFeltFloodMaskInv)
ret,imageWoodRailsMask = cv.threshold(imageWoodRailsMask,127,255,cv.THRESH_BINARY)

# Apply the mask to the original image using bitwise-and
imageWoodRailsMaskRGB = cv.cvtColor(imageWoodRailsMask, cv.COLOR_GRAY2BGR)
ret,imageWoodRailsMaskRGB = cv.threshold(imageWoodRailsMaskRGB,127,255,cv.THRESH_BINARY)
imageWoodRails = cv.bitwise_and(imageOriginal, imageWoodRailsMaskRGB)

ImgShow([img, imageDrawnLines, imageConvexMask, imageWoodRailsMask, imageWoodRails],220)

In [ ]:
# Hough Circles. 
imageInput = imageWoodRails
print("imageInput.shape = {0}".format(imageInput.shape))

# Limit by size range and distance
cols = imageInput.shape[1]
rows = imageInput.shape[0]
imageGray = cv.cvtColor(imageInput, cv.COLOR_BGR2GRAY)
imageGray = cv.GaussianBlur(imageGray, (7,7), 2,2)
hcircles = cv.HoughCircles(imageGray, cv.HOUGH_GRADIENT, minDist=60, dp=1,
                               param1=100, param2=10,
                               minRadius=5, maxRadius=30)

# Create an array of points
markerPoints = []
imageCircles = imageOriginal.copy()
if hcircles is not None:
    hcircles = np.uint16(np.around(hcircles))
    for i in hcircles[0, :]: 
        pt = [i[0], i[1]]
        radius = i[2]
    
        # Filter out points ouside of image
        if pt[0] < 0 or pt[1] < 0:
            continue
        if pt[0] > cols-1 or pt[1] > rows-1:
            continue
    
        markerPoints.append(pt)
        cv.circle(imageCircles, pt, radius, Color(), 7)
        
print("markerPoints count = {0}".format(len(markerPoints)))
    
ImgShow([imageInput, imageCircles], 320)

In [ ]:
# Filter out circles that don't match the color we want
# We will assume that the circle centers should be whiteish

# Find the range
clrWhite = (240, 240, 240)
fuzz = 45

imageCirclesWhite = imageOriginal.copy()
imageCirclesWhiteBlack = np.zeros(imageOriginal.shape[:3],np.uint8)

clrMin, clrMax = ColorRange(clrWhite, fuzz)
clrMin = (clrMin[0]-1,clrMin[1]-1,clrMin[2]-1,)
clrMax = (clrMax[0]+1,clrMax[1]+1,clrMax[2]+1,)
print("color range = {0} - {1}".format(clrMin, clrMax))
circlesWhite = []
circRadius = 19
for pt in markerPoints:    
    clr = imageOriginal[pt[1],pt[0]]
    if (clr > clrMin).all() and (clr < clrMax).all():
        circlesWhite.append(pt)
        clr = Color()
        cv.circle(imageCirclesWhite, pt, circRadius, clr, 6)
        cv.circle(imageCirclesWhiteBlack, pt, circRadius, clr, 6)
        #print("pt={0} clr={1}".format(pt, clr))
print("Marker Prunage: markerPoints count={0}, circlesWhite count={1}".format(len(markerPoints), len(circlesWhite)))

ImgShow([imageCirclesWhite, imageCirclesWhiteBlack], 220)

In [ ]:
# Group the markers into lines

# Find the top and bottom points by scanning all points
ymin = 10E10
ymax = 0
for pt in circlesWhite:
    if pt[1] < ymin:
        ymin = pt[1]
    if pt[1] > ymax:
        ymax = pt[1]

# Sort the markers
markersL = []
markersR = []
markersT = []
markersB = []

yFuzz = int(imageOriginal.shape[0] / 20)    # search range
xmid = int(imageOriginal.shape[1] / 2)
for pt in circlesWhite:
    if pt[1] < ymin + yFuzz:
        markersT.append(pt)
    else:
        if pt[1] > ymax - yFuzz:
            markersB.append(pt)
        else:
            if pt[0] < xmid:
                markersL.append(pt)
            else:
                markersR.append(pt)

# Sort by position, helpful for imputing new phantom markers later on
markersL.sort(key = lambda circ: circ[1])
markersR.sort(key = lambda circ: circ[1])
markersT.sort(key = lambda circ: circ[0])
markersB.sort(key = lambda circ: circ[0])

# Another view into it
markersTLBR = [markersT, markersL, markersB, markersR]

print("markersL", len(markersL), markersL)
print("markersR", len(markersR), markersR)
print("markersT", len(markersT), markersT)
print("markersB", len(markersB), markersB)

In [ ]:
# Show on an image just for sanity
imageMarkerLines = imageOriginal.copy()

def DrawMarkers(img, mks):
    pt1 = mks[0]
    pt2 = mks[len(mks) - 1]
    
    clrMark=Color()
    clrLine = (0,0,255)
    
    # int
    pt1 = (int(pt1[0]), int(pt1[1]))
    pt2 = (int(pt2[0]), int(pt2[1]))
    
    cv.line(img, pt1, pt2, clrLine, 4)
    clr
    for pt in mks:
        pt = (int(pt[0]), int(pt[1]))
        #print(pt)
        cv.circle(img, pt, 25, clrMark, 4)
        
DrawMarkers(imageMarkerLines, markersL)
DrawMarkers(imageMarkerLines, markersR)
DrawMarkers(imageMarkerLines, markersT)
DrawMarkers(imageMarkerLines, markersB)
            
ImgShow([imageMarkerLines], 210)

In [87]:
# Find points on the lines
# There will likely be some points that don't belong, use linear regression line fitting
# to filter those out
import numpy.polynomial.polynomial as poly

def GetFit(pts):
    #print("GetFit")
    data_x = pts[:,0]
    data_y = pts[:,1]
    ret = np.polyfit(data_x, data_y, 1, full=True)
    coeffs, resid, rank, singular_values, rcond = ret
    if 2 == len(pts):
        resid = [0.0]   # Special case, we always get a fit with just two points
        
    # Convert the residual to an average distance. It is currently sum of the square of the distances.
    r = resid[0]
    dAve = math.sqrt(r / len(data_x))
    #print(f'r={r} dAve={dAve}')
    return coeffs, dAve

# Try until we find a great fit or fail (RECURSIVE!)    
def FindLinePoints(pts, rangeSize):    
    # Convert to numpy array if needed
    if 'numpy.ndarray' != type(pts):
        pts = np.array(pts, dtype=np.int32)
    cnt = len(pts)
    #print(f'FindLinePoints minPoints={minPoints} cnt={cnt}')
    #print(pts)   
    
    # Prune the top-level sets down to the max size, we won't search
    # point sets larger than the max size
    maxPoints = rangeSize[1]
    subsets = [pts]
    while True:
        for ptsOne in pts:
            if len(ptsOne) > maxPoints:
                # Set is too big, remove one point in all combinations
                for i in range(0, len(ptsOne)):
                ptsMinusOne = np.delete(ptsOne, i, 0)
                subsets.append(ptsMinusOne)
    
        
    # The order of the search matters (we want the largest set),
    # so we delay recursion until we have processed all top-level
    # sets. We play a little trick here by inserting the 'full' set
    # at the -1 index.
    subsets = []
    for i in range(-1, len(pts)):
        if -1 == i:
            ptsLess = pts
        else:
            ptsLess = np.delete(pts, i, 0)
            subsets.append(ptsLess)
        coeffs, aveDist = GetFit(ptsLess)
        #print(f' i={i} aveDist={aveDist}')
        if aveDist < maxAveDist:
            #print(" found")
            return ptsLess  # Yes, we have a nice fit
    
    # We don't have a nice fit, remove a point and get recursive
      
    # Are we at the end of our search?
    if len(pts) == minPoints:
        return []
    for subset in subsets:
        ptsFit = FindLinePoints(subset, minPoints, maxAveDist)
        if len(ptsFit) > 0:
            return ptsFit   # Found a line
        
    # Did not find it
    return []


def ProcessLine(name, pts, minPoints, maxAveDist, process=True):
    if not process:
        return pts
    ptsFit = FindLinePoints(pts, minPoints, maxAveDist)
    cntB = len(pts)
    cntA = len(ptsFit)
    #print(f'name={name} minPoints={minPoints} maxAveDist={maxAveDist} len: {cntB}->{cntA}')
    return ptsFit



maxAveDist = 375.0
#markersR2 = ProcessLine("markersR2", markersR, minPoints=6, maxAveDist=maxAveDist)
#markersL2 = ProcessLine("markersL2", markersL, minPoints=6, maxAveDist=maxAveDist)
#markersT2 = ProcessLine("markersT2", markersT, minPoints=2, maxAveDist=maxAveDist)
#markersB2 = ProcessLine("markersB2", markersB, minPoints=2, maxAveDist=maxAveDist)
    



In [ ]:
def GetThem():
    return 1,2,3,4,5

a, *b, c = GetThem()
print(f'Unpacking:')
print(f'  a={a} b={b} c={c}')

print("enumerate demo")
names = ["bob", "jenny", "Brendan"]
nicknames = ["throb", "slutty jen", "Honey Badger"]
for idx, name in enumerate(names):
    nickname = nicknames[idx]
    print(f'  idx={idx} name={name} nickname={nickname}')

print("zip demo")
names = ["bob", "jenny", "Brendan"]
nicknames = ["throb", "slutty jen", "Honey Badger"]
ages = [22, 35, 39]
for name, nickname, age in zip(names, nicknames, ages):
    print(f'  name={name} nickname={nickname} age={age}')

In [ ]:
# Constructing output list WITHOUT
# Using List comprehensions
input_list = [1, 2, 3, 4, 4, 5, 6, 7, 7]

output_list = []

# Using loop for constructing output list
for var in input_list:
	if var % 2 == 0:
		output_list.append(var)

print("Output List using for loop:", output_list)

# ----------------------------------------------
# Using List comprehensions
# for constructing output list



list_using_comp = [var for var in input_list if var % 2 == 0]

print("Output List using list comprehensions:",
							list_using_comp)

array = [1, "two", 3]
tup = (1, "two", 3)
print(array, tup)


In [ ]:
# Find the corner points of intersection

def FindLine(mks):
    pt1 = mks[0]
    pt2 = mks[len(mks) - 1]
    x1 = pt1[0]
    y1 = pt1[1]
    x2 = pt2[0]
    y2 = pt2[1]
    rise = float(float(y2) - float(y1))
    run  = float(float(x2) - float(x1))
    if run != 0.0:
        m = rise/run
    else:
        m = float(10E10)
    b = float(y1 - m * x1)
    return (m,b)


def FindIntersection(mksA, mksB):
    lineA = FindLine(mksA)
    lineB = FindLine(mksB)
    a = lineA[0]
    c = lineA[1]
    b = lineB[0]
    d = lineB[1]
    
    x = (d-c)/(a-b)
    y = a*x+c
    return (round(x),round(y))


ptTL = FindIntersection(markersL, markersT)
ptTR = FindIntersection(markersT, markersR)
ptBL = FindIntersection(markersL, markersB)
ptBR = FindIntersection(markersR, markersB)

imageCorners = imageMarkerLines.copy()
print(ptTL)
dia = 175
clrCorner = (0,0,255)
fill = 10
cv.circle(imageCorners, ptTL, dia, clrCorner, fill)
cv.circle(imageCorners, ptTR, dia, clrCorner, fill)
print(ptBL)
cv.circle(imageCorners, ptBL, dia, clrCorner, fill)
cv.circle(imageCorners, ptBR, dia, clrCorner, fill)

ImgShow([imageCorners], 200)

In [ ]:
# Perspective Transform
rows = imageOriginal.shape[0]
cols = imageOriginal.shape[1]

# Convert a scalar point to an array
def ToPtArray(ptIn):
    ptsOut = []
    x,y = ptIn
    ptsOut = [x,y]
    return ptsOut

# The transformed space should be in dimensions that have the 
# correct aspect ratio for a pool table (2:1)
if rows > cols:
    tgtDims = (cols, cols*2)
else:
    tgtDims = (rows*2, rows)
print("Target Dims = {0}".format(tgtDims))

# Build the array of src and dst points
ptsTransformSrc = np.array( [
    ToPtArray(ptTL),
    ToPtArray(ptTR), 
    ToPtArray(ptBR),
    ToPtArray(ptBL) ]
         ).astype(np.float32)

ptsTransformDst = np.array( [    
    [0,0],
    [tgtDims[0]-1, 0],
    [tgtDims[0]-1, tgtDims[1]-1],
    [0, tgtDims[1]-1]] 
         ).astype(np.float32)

# Calculate the transform matrix
matTxToFlat = cv.getPerspectiveTransform(ptsTransformSrc, ptsTransformDst)
matTxFromFlat = cv.getPerspectiveTransform(ptsTransformDst, ptsTransformSrc)


# Apply the warp transform
imageWarp = cv.warpPerspective(imageCorners, matTxToFlat, tgtDims)

print("Perspective Transform")
ImgShow([imageCorners, imageWarp],240)

In [ ]:
# Transform functions

def NPA(inp):
    # Convert to float array
    rows = []
    for row in inp:
        rowArray = []
        for c in row:
            rowArray.append(float(c))
        rows.append(rowArray)
        
    # Build a numpy array
    return np.array(rows).astype(np.float32)

def TxPoint(matTx, pt):
    c3 = matTx[2,0]*pt[0] + matTx[2,1]*pt[1] + matTx[2,2]
    m = [(matTx[0,0]*pt[0] + matTx[0,1]*pt[1] + matTx[0,2])/c3,
         (matTx[1,0]*pt[0] + matTx[1,1]*pt[1] + matTx[1,2])/c3]
    return m

def TxPoints(matTx, pts):
    out = []
    for pt in pts:
        a = TxPoint(matTx, pt)
        out.append(a)
    return out

matTx = NPA([[2,0.5,-100],
            [0,2,0],
            [0,0.005,1]])

ptTx = TxPoint(matTx, [100.0, 100.0])
print("ptTx",ptTx)
X = TxPoints(matTx, 
                [[100.0, 100.0],
                 [100.0, 100.0],
                 [100.0, 100.0],
                 [100.0, 100.0]])
print("X",X)
# Should be [100.00000074505806, 133.33333432674408]]

# We can go both ways, test to be sure
pt =  [500,1000]
print("pt", [500,1000])
ptTx = TxPoint(matTxToFlat, pt)
ptOrig = TxPoint(matTxFromFlat, ptTx)
print("ptFlat",ptTx)
print("ptOrig",ptOrig)

In [ ]:
import urllib
import cv2 as cv

url = "https://www.cameraftp.com/cameraftp/publish/Camera.aspx/parentID197686853/shareID13469020/Sunset"
rstp = "rtsp://wowzaec2demo.streamlock.net/vod/mp4:BigBuckBunny_115k.mp4"
cap = cv.VideoCapture(rstp)
while True:
    ret,frame=cap.read()
    cv.imshow("RTSP",frame)
    k = cv.waitKey(1)
    if( k == ord('q')):
        break
    
#resp = urllib.urlopen(url)
#img = np.array(bytearray(resp.read))